In [31]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from math import *
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
import pandas as pd
from sklearn.inspection import permutation_importance


In [32]:
train_df = pd.read_csv('../data/raw/dmml1_train.csv')
test_df = pd.read_csv('../data/raw/dmml1_test.csv')
store = pd.read_csv('../data/raw/dmml1_stores.csv')

In [33]:
train_df = pd.merge(train_df,store,how="inner",on="Store ID")
test_df = pd.merge(test_df,store,how="inner",on="Store ID")

In [34]:
train_df

,Store ID,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,4,2015-04-30,6884,716,1,1,0,0,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
1,1,3,2015-04-29,6764,756,1,1,0,0,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,1,2,2015-04-28,6861,678,1,1,0,0,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
3,1,1,2015-04-27,6523,647,1,1,0,0,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
4,1,7,2015-04-26,0,0,0,0,0,0,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246898,300,6,2013-01-05,5194,569,1,0,0,0,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
246899,300,5,2013-01-04,5524,646,1,0,0,1,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
246900,300,4,2013-01-03,5563,718,1,0,0,1,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
246901,300,3,2013-01-02,6218,753,1,0,0,1,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN


In [35]:
store

,Store ID,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
1,2,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
2,3,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
3,4,a,c,1070.0,NaN,NaN,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
4,5,a,c,4590.0,3.0,2000.0,1,40.0,2011.0,"Jan,Apr,Jul,Oct"
...,...,...,...,...,...,...,...,...,...,...
295,296,a,a,690.0,6.0,2007.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
296,297,b,b,720.0,3.0,2002.0,0,NaN,NaN,NaN
297,298,d,c,1340.0,10.0,2006.0,1,5.0,2013.0,"Feb,May,Aug,Nov"
298,299,d,a,260.0,2.0,2012.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"


In [36]:
train_df["CompetitionDistance"].replace(np.nan,train_df["CompetitionDistance"].mean(),inplace=True)
train_df.fillna(0,inplace=True)

In [37]:
train_df["StateHoliday"].replace("0",0,inplace=True)

In [38]:
train_df['Date'] = pd.to_datetime(train_df['Date'])
train_df['Year'] = train_df['Date'].dt.year
train_df['Month'] = train_df['Date'].dt.month
train_df['Day'] = train_df['Date'].dt.day
train_df['WeekOfYear'] = train_df['Date'].dt.isocalendar().week
train_df['Weekend'] = np.where(train_df['DayOfWeek'].isin([6, 7]), 1, 0)  # Samstag = 6, Sonntag = 7

train_df['Quarter'] = train_df['Date'].dt.quarter
train_df['DayOfYear'] = train_df['Date'].dt.dayofyear
train_df['DayOfMonth'] = train_df['Date'].dt.day

train_df['Season'] = train_df['Month'].apply(lambda month: (month%12 // 3 + 1))
train_df['Season'].replace(to_replace=[1,2,3,4], value=['Winter', 'Frühling','Sommer','Herbst'], inplace=True)

# Entfernen der ursprünglichen Date-Spalte
train_df.drop('Date', axis=1, inplace=True)

In [39]:
train_df["StateHoliday"] = train_df["StateHoliday"].replace("a","Feiertag").replace("b","Ostern").replace("c","Weihnachten").replace(0,"Kein Feiertag")

In [40]:
train_df["StoreType"] = train_df["StoreType"].replace("a","StoreType A").replace("b","StoreTypee B").replace("c","StoreType C").replace("d","StoreType D")

In [41]:
train_df["Assortment"] = train_df["Assortment"].replace("a","Basic").replace("b","Extra").replace("c","Extended")

In [42]:
# train_df["PromoInterval"] = train_df["PromoInterval"].replace(0,"No PromoInterval").replace("Jan,Apr,Jul,Oct","From Jan to Oct").replace("Feb,May,Aug,Nov","From Feb to Nov").replace("Mar,Jun,Sept,Dec","From Mar to Dec")

def myPinterval(x):
    if x=='From Feb to Nov':
        return([0,1,0,0,1,0,0,1,0,0,1,0])
    elif x=='From Jan to Oct':
        return([1,0,0,1,0,0,1,0,0,1,0,0])
    elif x== 'From Mar to Dec':
        return([0,0,1,0,0,1,0,0,1,0,0,1])
    else:
        return(np.repeat(0,12).tolist())

proInt = train_df.PromoInterval.apply(myPinterval).tolist()
proInt = pd.DataFrame(proInt, columns = ['PromoInt'+ str(i) for i in range(1,13)])
train_df = train_df.drop('PromoInterval',1).join(proInt)

/var/folders/jq/xvfn2vg55lsc1wx2_2mrh3640000gn/T/ipykernel_57679/406701526.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train_df = train_df.drop('PromoInterval',1).join(proInt)


In [43]:
# Erstellen Sie eine temporäre Spalte, die 1 ist, wenn der Tag ein Feiertag ist, und 0 sonst
train_df['Temp'] = np.where(train_df['StateHoliday'] != 'Kein Feiertag', 1, 0)

# Erstellen Sie die Spalte "TagVorFeiertag", die 1 ist, wenn der nächste Tag ein Feiertag ist, und 0 sonst
train_df['TagVorFeiertag'] = train_df['Temp'].shift(-1).fillna(0)

# Erstellen Sie die Spalte "TagnachFeiertag", die 1 ist, wenn der vorherige Tag ein Feiertag war, und 0 sonst
train_df['TagnachFeiertag'] = train_df['Temp'].shift(1).fillna(0)

# Entfernen Sie die temporäre Spalte
train_df.drop('Temp', axis=1, inplace=True)

In [44]:
# Erstellen Sie den OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Liste der Spalten, die encodiert werden sollen
columns_to_encode = ['StateHoliday', 'StoreType', 'Assortment', 'Season']

for column in columns_to_encode:
    # Führen Sie das One-Hot-Encoding für die Spalte durch
    column_encoded = encoder.fit_transform(train_df[[column]])

    # Erstellen Sie einen DataFrame aus dem encodierten Array
    column_df = pd.DataFrame(column_encoded, columns=encoder.categories_[0])

    # Fügen Sie die neuen Spalten zum DataFrame hinzu und entfernen Sie die alte Spalte
    train_df = pd.concat([train_df.drop(column, axis=1), column_df], axis=1)

In [45]:
X = train_df.drop(["Sales","Customers"],axis=1)
y = train_df["Sales"]

In [46]:
X

,Store ID,DayOfWeek,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,...,StoreType C,StoreType D,StoreTypee B,Basic,Extended,Extra,Frühling,Herbst,Sommer,Winter
0,1,4,1,1,0,570.0,11.0,2007.0,1,13.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,3,1,1,0,570.0,11.0,2007.0,1,13.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,2,1,1,0,570.0,11.0,2007.0,1,13.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,1,1,1,0,570.0,11.0,2007.0,1,13.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1,7,0,0,0,570.0,11.0,2007.0,1,13.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246898,300,6,1,0,0,9260.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
246899,300,5,1,0,1,9260.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
246900,300,4,1,0,1,9260.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
246901,300,3,1,0,1,9260.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [47]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [49]:
forest = RandomForestRegressor(random_state=42)
forest.fit(X_train, y_train)
importances = forest.feature_importances_

array([7.26984228e-02, 2.89458080e-02, 4.32604880e-01, 6.45206193e-02,
       2.60338503e-03, 1.50596379e-01, 4.18765909e-02, 2.99549053e-02,
       7.08858764e-03, 1.33883275e-02, 3.08251094e-02, 6.65032371e-03,
       2.22934148e-03, 9.01014227e-03, 8.89127423e-03, 4.59997239e-03,
       6.54949155e-04, 2.21121404e-02, 8.88339586e-03, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.74664705e-03,
       4.54204221e-03, 3.79617704e-05, 3.38590294e-04, 1.51937396e-05,
       2.02124791e-06, 5.89781232e-03, 9.75805597e-03, 4.80177068e-03,
       2.94541341e-03, 1.47805727e-02, 1.09677644e-02, 1.42028856e-04,
       9.51008782e-04, 3.14258626e-04, 1.26086101e-03, 3.63442360e-04])

In [50]:
columns = list(X_train.columns)
columns
feature_importance_df = pd.DataFrame({'Features':columns, 'Values':importances})
feature_importance_df.sort_values(by='Values', ascending=False, inplace=True)


In [51]:
model_spalten = feature_importance_df.loc[feature_importance_df['Values'] > 0.01].Features.tolist()

In [67]:
model_spalten

['Open',
 'CompetitionDistance',
 'Store ID',
 'Promo',
 'CompetitionOpenSinceMonth',
 'Promo2SinceYear',
 'CompetitionOpenSinceYear',
 'DayOfWeek',
 'DayOfYear',
 'Basic',
 'Promo2SinceWeek',
 'Extended']

In [61]:
len(model_spalten)

12

In [53]:
X = train_df[model_spalten]
y = train_df["Sales"]

In [65]:
X

,Open,CompetitionDistance,Store ID,Promo,CompetitionOpenSinceMonth,Promo2SinceYear,CompetitionOpenSinceYear,DayOfWeek,DayOfYear,Basic,Promo2SinceWeek,Extended
0,1,570.0,1,1,11.0,2010.0,2007.0,4,120,1.0,13.0,0.0
1,1,570.0,1,1,11.0,2010.0,2007.0,3,119,1.0,13.0,0.0
2,1,570.0,1,1,11.0,2010.0,2007.0,2,118,1.0,13.0,0.0
3,1,570.0,1,1,11.0,2010.0,2007.0,1,117,1.0,13.0,0.0
4,0,570.0,1,0,11.0,2010.0,2007.0,7,116,1.0,13.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
246898,1,9260.0,300,0,0.0,0.0,0.0,6,5,0.0,0.0,1.0
246899,1,9260.0,300,0,0.0,0.0,0.0,5,4,0.0,0.0,1.0
246900,1,9260.0,300,0,0.0,0.0,0.0,4,3,0.0,0.0,1.0
246901,1,9260.0,300,0,0.0,0.0,0.0,3,2,0.0,0.0,1.0


In [56]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [57]:
models=[LinearRegression(), Ridge(), Lasso(), DecisionTreeRegressor(random_state=42), GradientBoostingRegressor(random_state=42), RandomForestRegressor(n_estimators=42, n_jobs=-1, random_state=42)]
model_names=["LinearRegression","Ridge","Lasso","RandomForestRegressor","DecisionTreeRegressor","GradientBoostingRegressor"]

MSE=[]
r_2=[]
accuracy = []
RMSE = []
training_time = []
d={}

for index in range (len(models)):
    model=models[index]
    start_time = time.time()
    model.fit(X_train,y_train)
    end_time = time.time()
    y_pre=model.predict(X_test)

    MSE.append(round(mean_squared_error(y_true=y_test,y_pred=y_pre),5))
    r_2.append(r2_score(y_true=y_test,y_pred=y_pre))
    accuracy.append((model.score(X_test,y_test))*100)
    RMSE.append(sqrt(mean_squared_error(y_true=y_test,y_pred=y_pre)))
    training_time.append(end_time - start_time)
    print(f'{model_names[index]} wurde trainiert und benötigte {end_time - start_time} Sekunden')

d=pd.DataFrame({'Modelling Name':model_names,'MSE':MSE,"R_2":r_2,"Accuracy":accuracy,"RMSE":RMSE, "Training Time": training_time})
print("Verwendete Features: ", X_train.columns.tolist())

LinearRegression wurde trainiert und benötigte 0.2835416793823242 Sekunden
Ridge wurde trainiert und benötigte 0.014171838760375977 Sekunden
Lasso wurde trainiert und benötigte 0.5106239318847656 Sekunden
RandomForestRegressor wurde trainiert und benötigte 0.57289719581604 Sekunden
DecisionTreeRegressor wurde trainiert und benötigte 12.88463306427002 Sekunden
GradientBoostingRegressor wurde trainiert und benötigte 3.508575916290283 Sekunden
Verwendete Features:  ['Open', 'CompetitionDistance', 'Store ID', 'Promo', 'CompetitionOpenSinceMonth', 'Promo2SinceYear', 'CompetitionOpenSinceYear', 'DayOfWeek', 'DayOfYear', 'Basic', 'Promo2SinceWeek', 'Extended']


In [62]:
d

,Modelling Name,MSE,R_2,Accuracy,RMSE,Training Time
0,LinearRegression,7.430315e+06,0.540504,54.050401,2725.860506,0.283542
1,Ridge,7.430321e+06,0.540504,54.050364,2725.861592,0.014172
2,Lasso,7.432120e+06,0.540392,54.039243,2726.191444,0.510624
3,RandomForestRegressor,1.543403e+06,0.904555,90.455486,1242.337756,0.572897
4,DecisionTreeRegressor,4.670201e+06,0.711191,71.119144,2161.064820,12.884633
5,GradientBoostingRegressor,1.018490e+06,0.937016,93.701584,1009.202809,3.508576


In [66]:
# Definieren Sie die Parameter, die Sie optimieren möchten
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Erstellen Sie ein neues RandomForestRegressor-Objekt
forest = RandomForestRegressor(random_state=42)

# Erstellen Sie das GridSearchCV-Objekt
grid_search = GridSearchCV(forest, param_grid, cv=5, n_jobs=-1)

# Führen Sie die Suche aus
grid_search.fit(X_train, y_train)

# Drucken Sie die besten Parameter aus
print(grid_search.best_params_)


/Users/wladi/opt/anaconda3/envs/bv/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'max_depth': None, 'min_samples_split': 10, 'n_estimators': 200}


{'max_depth': None, 'min_samples_split': 10, 'n_estimators': 200}

In [68]:
# Definieren Sie die optimierten Parameter
best_params = {'max_depth': None, 'min_samples_split': 10, 'n_estimators': 200}

# Erstellen Sie ein neues RandomForestRegressor-Objekt mit den optimierten Parametern
forest = RandomForestRegressor(max_depth=best_params['max_depth'], 
                               min_samples_split=best_params['min_samples_split'], 
                               n_estimators=best_params['n_estimators'],
                               random_state=42)

# Trainieren Sie das Modell mit den Trainingsdaten
forest.fit(X_train, y_train)

RandomForestRegressor(min_samples_split=10, n_estimators=200, random_state=42)

In [69]:
# Machen Sie Vorhersagen auf den Testdaten
y_pred = forest.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # RMSE ist einfach die Quadratwurzel von MSE
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² Score: {r2}")

Mean Squared Error (MSE): 945929.6137281588
Root Mean Squared Error (RMSE): 972.5891289378875
R² Score: 0.9415030404690825
